<a href="https://colab.research.google.com/github/Heffx/CNN/blob/master/Projeto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
# Conferindo se o CUDA esta disponivel

verificaGPU = torch.cuda.is_available()

if not verificaGPU:
  print('CUDA INDISPONIVEL, TREINANDO COM CPU')
else:
  print('CUDA DISPONIVEL, TREINANDO COM GPU')

CUDA DISPONIVEL, TREINANDO COM GPU


In [3]:
!pip3 install torchvision

In [4]:
# Pillow
!pip install Pillow==4.1.1

!pip install image

In [0]:
# Fazendo download e descompactando o dataset
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!rm -r flower_data || true
!unzip -qq flower_data.zip

In [0]:
import time
import json
import copy
import flower_data
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import PIL
from PIL import Image
from collections import OrderedDict
import torch
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as F
import glob
import os
from skimage import data, io, filters
from scipy.optimize import minimize


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Carregando, Normalizando e Criando vetor de Imagens

In [8]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        # Redimensiona a imagem em (224,224)
        transforms.RandomResizedCrop(224),
        # Inverte horizontalmente a imagem
        transforms.RandomHorizontalFlip(),
        # Transforma em matriz de dados unicos
        transforms.ToTensor(),
        # Normaliza as imagens atraves de desvio padrão(Valores retirados de outro trabalho) 
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        # Transformação da imagem para tamanho de entrada
        transforms.Resize(256),
        # Recorta a imagem no centro
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


print(data_transforms)

{'train': Compose(
    RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=PIL.Image.BILINEAR)
    RandomHorizontalFlip(p=0.5)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
), 'valid': Compose(
    Resize(size=256, interpolation=PIL.Image.BILINEAR)
    CenterCrop(size=(224, 224))
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)}


In [0]:
train_dir = 'flower_data/train/'
valid_dir = 'flower_data/valid/'
dirs = {'train': train_dir, 
        'valid': valid_dir, 
       }

In [0]:
image_datasets = {x: datasets.ImageFolder(dirs[x],
                                          transform = data_transforms[x])
                  for x in ['train', 'valid']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                              batch_size=32, shuffle=True) 
               for x in ['train', 'valid']}
dataset_sizes = {x: len(image_datasets[x]) 
                              for x in ['train', 'valid']}
class_names = image_datasets['train'].classes

In [13]:
print(dataset_sizes)
print(device)

{'train': 6552, 'valid': 818}
cuda:0


In [0]:
# Label mapping
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

In [16]:
images, labels = next(iter(dataloaders['train']))
images.size()
images, labels = next(iter(dataloaders['train']))
rand_idx = np.random.randint(len(images))
# print(rand_idx)
print("label: {}, class: {}, name: {}".format(labels[rand_idx].item(),
                                               class_names[labels[rand_idx].item()],
                                               cat_to_name[class_names[labels[rand_idx].item()]]))

label: 90, class: 89, name: watercress


## Pre-treino

In [0]:
model = models.vgg19(pretrained=True)

In [18]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (17): ReLU(inplace)

In [0]:
#Definindo a arquitetura da rede

class Classifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.fc1 = nn.Linear(25088,4096)
    self.fc2 = nn.Linear(4096,102)
    
  def forward(self,x):
    #verifica se o tensor está "flattened (achatado)"
    x = x.view(x.shape[0],-1)
    
    x = F.relu(self.fc1(x))
    x = F.log_softmax(self.fc2(x),dim=1)
    
    return x

In [0]:
classifier = Classifier()
for param in model.parameters():
    param.requires_grad = False

In [0]:
model.classifier = classifier

In [0]:
if verificaGPU:
    model.cuda()

In [0]:
# Criteria NLLLoss which is recommended with Softmax final layer
criterion = nn.NLLLoss()
# Observe that all parameters are being optimized
optim = torch.optim.Adam(model.classifier.parameters(), lr=0.001)
# Decay LR by a factor of 0.1 every 4 epochs
sched = lr_scheduler.StepLR(optim, step_size=4, gamma=0.1)
# Number of epochs
eps=10

In [0]:
def train_model(model, criteria, optimizer, scheduler,    
                                      num_epochs=25, device='cuda'):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [30]:
model_ft = train_model(model, criteria, optim, sched, eps, 'cuda')

Epoch 0/9
----------
train Loss: 2.1997 Acc: 0.5082
valid Loss: 0.7873 Acc: 0.7861

Epoch 1/9
----------
train Loss: 0.9887 Acc: 0.7308
valid Loss: 0.7072 Acc: 0.7995

Epoch 2/9
----------
train Loss: 0.8069 Acc: 0.7785
valid Loss: 0.5680 Acc: 0.8447

Epoch 3/9
----------
train Loss: 0.7787 Acc: 0.7879
valid Loss: 0.5735 Acc: 0.8496

Epoch 4/9
----------
train Loss: 0.5132 Acc: 0.8593
valid Loss: 0.3992 Acc: 0.8912

Epoch 5/9
----------
train Loss: 0.4330 Acc: 0.8829
valid Loss: 0.3754 Acc: 0.8961

Epoch 6/9
----------
train Loss: 0.3945 Acc: 0.8918
valid Loss: 0.3597 Acc: 0.9071

Epoch 7/9
----------
train Loss: 0.3784 Acc: 0.8939
valid Loss: 0.3687 Acc: 0.9071

Epoch 8/9
----------
train Loss: 0.3453 Acc: 0.9058
valid Loss: 0.3564 Acc: 0.9059

Epoch 9/9
----------
train Loss: 0.3575 Acc: 0.9006
valid Loss: 0.3530 Acc: 0.9059

Training complete in 36m 7s
Best val Acc: 0.907090


In [0]:
# Função pronta # Em estudo
def calc_accuracy(model, data, cuda=False):
    model.eval()
    model.to(device='cuda')    
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(dataloaders[data]):
            if cuda:
                inputs, labels = inputs.cuda(), labels.cuda()
            # obtain the outputs from the model
            outputs = model.forward(inputs)
            # max provides the (maximum probability, max value)
            _, predicted = outputs.max(dim=1)
            # check the 
            if idx == 0:
                print(predicted) #the predicted class
                print(torch.exp(_)) # the predicted probability
            equals = predicted == labels.data
            if idx == 0:
                print(equals)
            print(equals.float().mean())

In [42]:
acuracia = calc_accuracy(model, 'valid', True)

tensor([ 45,  49,  65,  26,  64,  59,  30,  43,  50,  53,  11,  12,  43, 100,
         47,  51,  74,  99,  43,  39,  38,  11,  12,  26,  83,  38,  17,  89,
         82,  62,  38,  76], device='cuda:0')
tensor([0.9597, 0.9996, 0.9999, 0.9941, 1.0000, 1.0000, 0.9827, 0.3120, 0.6429,
        0.9978, 1.0000, 1.0000, 0.6562, 0.9970, 0.9925, 0.3685, 0.4184, 0.9832,
        0.9999, 0.9957, 0.6466, 1.0000, 0.9996, 0.9895, 0.6365, 0.9999, 0.9554,
        0.7745, 1.0000, 1.0000, 0.6963, 0.9577], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 0, 1, 1, 0, 1], device='cuda:0', dtype=torch.uint8)
tensor(0.8438, device='cuda:0')
tensor(0.8438, device='cuda:0')
tensor(0.9688, device='cuda:0')
tensor(0.9062, device='cuda:0')
tensor(0.8750, device='cuda:0')
tensor(0.9062, device='cuda:0')
tensor(0.8750, device='cuda:0')
tensor(0.9062, device='cuda:0')
tensor(0.9375, device='cuda:0')
tensor(0.8438, device='cuda:0')
tensor(0.8750, device='

NameError: ignored